In [173]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [174]:
torch.manual_seed(1)

In [175]:
LSTM = nn.LSTM(3,3)
input_ = [torch.randn(1,3) for _ in range(5)]
input_

[tensor([[-0.5525,  0.6355, -0.3968]]),
 tensor([[-0.6571, -1.6428,  0.9803]]),
 tensor([[-0.0421, -0.8206,  0.3133]]),
 tensor([[-1.1352,  0.3773, -0.2824]]),
 tensor([[-2.5667, -1.4303,  0.5009]])]

In [176]:
hidden = (
            torch.randn(1,1,3),
            torch.randn(1,1,3)
)
hidden

(tensor([[[ 0.5438, -0.4057,  1.1341]]]),
 tensor([[[-1.1115,  0.3501, -0.7703]]]))

In [177]:
for i in input_:
    out, hidden = LSTM(i.view(1,1,-1), hidden)

In [178]:
out

tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>)

In [179]:
hidden

(tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>),
 tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<StackBackward>))

In [180]:
input_ = torch.cat(input_).view(len(input_), 1,-1)
hidden = (torch.randn(1,1,3), torch.randn(1,1,3))

In [181]:
input_

tensor([[[-0.5525,  0.6355, -0.3968]],

        [[-0.6571, -1.6428,  0.9803]],

        [[-0.0421, -0.8206,  0.3133]],

        [[-1.1352,  0.3773, -0.2824]],

        [[-2.5667, -1.4303,  0.5009]]])

In [182]:
hidden

(tensor([[[-0.1473,  0.6272,  1.0935]]]),
 tensor([[[ 0.0939,  1.2381, -1.3459]]]))

PART OF SPEECH TAGGING

In [183]:
# output will be tensor([0, 1, 2, 3, 4, 5, 6, 7])
def prepare_sequence(seq, to_id):
    idex = [to_id[w] for w in seq]
    return torch.tensor(idex, dtype = torch.long)

In [184]:
train_data = [('There is a dog lying in the garden'.split(), ['DET','V','NUM', 'NN','V','PREP','DET','NN']),
              ('Today is a sunny day'.split(), ['NN','V','NUM','ADJ','NN'])
             ]
   

In [185]:
# define the unique word dict                                 
word_to_id = {}
for sent,tags in train_data:
    for word in sent:
        if word not in word_to_id:
            word_to_id[word] = len(word_to_id)
print(word_to_id)

{'There': 0, 'is': 1, 'a': 2, 'dog': 3, 'lying': 4, 'in': 5, 'the': 6, 'garden': 7, 'Today': 8, 'sunny': 9, 'day': 10}


In [186]:
# define tag id dict
tag_to_id = {"DET": 0, "NN": 1, "V": 2,'NUM':3,'PREP':4,'ADJ':5}

In [187]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

creat model

In [188]:
class LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
#         word_embedding is input, output is hidden state
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
    
        self.hidden_to_tag = nn.Linear(hidden_dim, tagset_size)
    
    def forward(self, sentence):
        embeds = self.word_embedding(sentence)
#         print('embeds are : ',embeds)
        lstm_out, _ = self.lstm(embeds.view(len(sentence),1,-1))
#         print('lstm_out and _ are here repectively： ', lstm_out, _)
        tag_space = self.hidden_to_tag(lstm_out.view(len(sentence), -1))
#         print('tag_space',tag_space)
        tag_score = F.log_softmax(tag_space, dim = 1)
#         print('tag_score is the output : ',tag_score)
        return tag_score

training process

In [189]:
print(len(word_to_id), len(tag_to_id))

11 6


In [190]:
model = LSTM(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_id), len(tag_to_id))
criterion = nn.NLLLoss()
optimizer = optim.SGD(filter(lambda p : p.requires_grad, model.parameters()), lr = 0.1)

In [191]:
with torch.no_grad():
    inputs = prepare_sequence(train_data[0][0], word_to_id)
#     print(inputs)
    tag_score = model(inputs)
#     print(tag_score)

In [192]:
word_to_id

{'There': 0,
 'is': 1,
 'a': 2,
 'dog': 3,
 'lying': 4,
 'in': 5,
 'the': 6,
 'garden': 7,
 'Today': 8,
 'sunny': 9,
 'day': 10}

In [205]:
for epoch in range(300):
    for sen, tag in train_data:
#         print(sen, tag)
        model.zero_grad()
        
        sen_input = prepare_sequence(sen, word_to_id)
#         print(sen_input)
        targets = prepare_sequence(tag, tag_to_id)
        
        tag_score = model(sen_input)
#         print(tag_score, targets)
        loss = criterion(tag_score, targets)
#         
        
        loss.backward()
        optimizer.step()
print(loss)

tensor(0.0624, grad_fn=<NllLossBackward>)


score after training

In [207]:
# Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.
with torch.no_grad():
    inputs = prepare_sequence(train_data[0][0], word_to_id)
    print(inputs)
    tag_scores = model(inputs)
    print(tag_scores)
ps = torch.exp(tag_scores).detach().numpy()
ps.shape

tensor([0, 1, 2, 3, 4, 5, 6, 7])
tensor([[-0.2784, -2.0590, -4.7458, -3.1336, -4.1343, -3.0549],
        [-4.9679, -4.4353, -0.0542, -4.7550, -3.6900, -7.8939],
        [-3.4356, -6.8892, -5.0189, -0.0676, -5.7163, -3.8069],
        [-4.3982, -0.0541, -5.0893, -6.6567, -3.7977, -4.5596],
        [-5.2755, -3.8170, -0.0634, -5.7282, -3.4806, -8.1740],
        [-3.2021, -2.9814, -2.4296, -4.3147, -0.2201, -5.3534],
        [-0.0736, -3.3402, -5.5987, -4.6752, -4.2477, -4.7992],
        [-3.1255, -0.0745, -4.8526, -6.1490, -4.2647, -5.5462]])


(8, 6)

In [193]:
tag_to_id

{'DET': 0, 'NN': 1, 'V': 2, 'NUM': 3, 'PREP': 4, 'ADJ': 5}

In [209]:
train_data[0][0]
# 02312401

['There', 'is', 'a', 'dog', 'lying', 'in', 'the', 'garden']

In [210]:
pred = [list(ps_).index(max(ps_)) for ps_ in ps]
pred

[0, 2, 3, 1, 2, 4, 0, 1]